# Testing Your Dev Environment

The purpose of this notebook is to ensure that your development environment for this workshop is setup correctly.

## Basic Libraries

Let's start by making sure all of the libraries are installed correctly:

In [1]:
import json
import outlines
from transformers import AutoTokenizer
import torch
from textwrap import dedent

## Loading models

This tutorial was built using `microsoft/Phi-3-medium-4k-instruct` but this can be too large for most laptops/notebook environments. Here are some other options you can try if this doesn't work:

- `microsoft/Phi-3-mini-4k-instruct`: Great choice for a fairly small model!
- `Qwen/Qwen2-0.5B-Instruct`: At 1GB this model should work on almost any (reasonable) machine

For doing the exercises the size of the model is not important, but you will get better outcomes with more powerful models.

In [2]:
model_name = "Qwen/Qwen2-0.5B-instructt"
model = outlines.models.transformers(
    model_name,
    model_kwargs={
        'trust_remote_code': True
    }
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

OSError: Qwen/Qwen2-0.5B-instructt is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
messages = [
    {
        "role": "assistant",
        "content": "I am a helpful AI, what would you like me to do?"
    },
    {
        "role": "user",
        "content": "Generate a phone number"
    },
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False)
prompt

## Unstructured Generation

In [ ]:
generator = outlines.generate.text(model)

In [ ]:
generator(prompt,max_tokens=24)

## Structured Generation

In [ ]:
generator_struct = outlines.generate.regex(model, r"[0-9]{3}-[0-9]{3}-[0-9]{4}")

In [ ]:
generator_struct(prompt)